# Password Checker IV

We have to be mindful of the fact that we're saving passwords in the command line's history. Not exactly the most secure option.
A good way to improve the program is to read the passwords from a text file, instead of a CLI argument.

Now, in line 35, we make sure that this file runs if it's the only file being run: `if __name__ == '__main__':`
This won't allow it to run in Jupyter, so we have to run it on the command line.

In [ ]:
import requests
import hashlib # for SHA-1 hashing
import sys


def request_api_data(query_char):
    url = 'https://api.pwnedpasswords.com/range/' + query_char
    res = requests.get(url)
    if res.status_code != 200:
      raise RuntimeError(f'Error fetching: {res.status_code}, check the API and try again')
    return res
  
def get_password_leaks_count(hashes, hash_to_check):
  hashes = (line.split(':') for line in hashes.text.splitlines())
  for h, count in hashes:
    if h == hash_to_check:
      return count
  return 0

def pwned_api_check(password):
    sha1password = hashlib.sha1(password.encode('utf-8')).hexdigest().upper()
    first5_char, tail = sha1password[:5], sha1password[5:]
    response = request_api_data(first5_char)
    return get_password_leaks_count(response, tail)
    
def main(args):
  for password in args:
    count = pwned_api_check(password)
    if count:
      print(f'{password} was found {count} times. Might want to change it')
    else:
      print(f'{password} was not found')
  return 'done!'

if __name__ == '__main__':
  sys.exit(main(sys.argv[1:])) # this makes the sys call exit and bring us back to the command line

Now, we have `done!` being returned with `sys.exit(main(sys.argv[1:]))` at the end. We're `exit`ing out of this file, so the entire `checkmypass.py` file was run, and the end of the function does indeed return `done!`.